This function assumed that we have an input file with 4 fields: id, time, lat and long.

In [95]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 

In [98]:
INPUT_PATH = "/Users/Etienne/Desktop/FaceAuVirus/confidential/sample_confidential_formatted.csv"
OUTPUT_PATH = "/Users/Etienne/Desktop/FaceAuVirus/confidential/cg_sample_confidential_formatted.csv"

df = pd.read_csv(INPUT_PATH)
df.head(5)

,time,id,lat,long
0,2020-03-11 10:20:13,10,49.426223,2.105698
1,2020-03-11 10:20:13,10,49.426223,2.105698
2,2020-03-11 10:30:33,10,49.426196,2.105919
3,2020-03-11 10:30:33,10,49.426196,2.105919
4,2020-03-11 10:41:20,10,49.426223,2.105710


With that kind of input files, you can then use the following function to build the graph.

In [99]:
def build_graph(input_csv, output_graph_csv):

    df = pd.read_csv(FILENAME, usecols=["time","id","lat","long"])
    df.time = pd.to_datetime(df.time)

    proximity_threshold = 0.000015 # (approxi 1m); Important assumption, check for reasonable values and always report values chosen
    time_threshold = 10 # In minutes. Important assumption, check for reasonable values and always report values chosen
    final_list = []
    for index, row in df.iterrows():

        person = row['id']
        time = row['time']
        lat = row['lat']
        long = row['long']

        # Get rows whose geographically and temporally close to the "actual row" (by proximity_threshold and time_threshold) 
        geo_time_slice = df[(df['lat'].between(lat - proximity_threshold, lat + proximity_threshold,inclusive=True)) &
                            (df['long'].between(long - proximity_threshold, long + proximity_threshold,inclusive=True)) &
                            (df['time'].between(time + timedelta(minutes =- time_threshold), time + timedelta(minutes =+ time_threshold),inclusive=True)) &
                            (df['id'] != person)]
        
        final_list.append([person, time.strftime("%Y-%m-%d %H:%M:%S"), lat, long, geo_time_slice['id'].unique().tolist()])

    df_contact = pd.DataFrame(final_list, columns = ['id', 'time', 'lat','long','contact_list'])
    df_contact.to_csv(output_graph_csv,  index=False)
    return


For example:

In [100]:
%%time
build_graph(INPUT_PATH, OUTPUT_PATH)

CPU times: user 3.59 s, sys: 6.22 ms, total: 3.6 s
Wall time: 3.6 s


# Expected result

If the build_graph function worked, it should give us a .csv file as the following.

In [94]:
df = pd.read_csv(OUTPUT_PATH)
df.head(10)

,id,time,lat,long,contact_list
0,10,2020-03-11 10:20:13,49.426223,2.105698,"[12, 13, 1, 4]"
1,10,2020-03-11 10:20:13,49.426223,2.105698,"[12, 13, 1, 4]"
2,10,2020-03-11 10:30:33,49.426196,2.105919,"[12, 13, 1, 4]"
3,10,2020-03-11 10:30:33,49.426196,2.105919,"[12, 13, 1, 4]"
4,10,2020-03-11 10:41:20,49.426222,2.105710,"[13, 1, 4]"
5,10,2020-03-11 10:41:20,49.426222,2.105710,"[13, 1, 4]"
6,10,2020-03-11 07:24:00,49.426222,2.105697,"[5, 4]"
7,10,2020-03-11 17:22:50,49.403637,2.176646,"[7, 4]"
8,10,2020-03-11 17:24:56,49.403630,2.176647,"[7, 4]"
9,10,2020-03-11 16:03:41,49.426215,2.105722,[4]


Again, we assume that, build_graph function is applied to a csv per day (name of each file is simply the date) and output is named as that same day, but with a prefix cg_